In [ ]:
#!pip install mxnet
#!pip install gluonts

import numpy as np
import pandas as pd
from pandas import datetime
import mxnet as mx
from math import sqrt
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

from gluonts.dataset.common import ListDataset
#from statsmodels.tsa import arima_model
from statsmodels.tsa.arima.model import ARIMA
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.npts import NPTSPredictor
from gluonts.mx.trainer import Trainer
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions 


In [3]:
cd Desktop

C:\Users\user\Desktop


## Prepare the data :

In [4]:
df = pd.read_csv('InputTimeSerie.csv', sep=';')

In [5]:
df.head()

,Id_Client,id_Product,Period,Cnt_Sale
0,45,21010760000000AM,202101,12
1,46,2517428PALU00100,202102,4991
2,50,21010150000000VE,202101,1152
3,50,21010150000000VE,202107,480
4,50,2570725000000000,202009,8800


In [6]:
df.Period = df.Period.apply(lambda x: str(x)[:4]+'-'+str(x)[4:])

df.Period = df.Period.apply(lambda x: datetime.strptime(str(x), '%Y-%m'))
df.tail()

,Id_Client,id_Product,Period,Cnt_Sale
3508,3781,41002530000000TR,2021-04-01,2000
3509,3829,2517328000000000,2021-07-01,80000
3510,3852,4202169CMAA001BL,2021-06-01,46
3511,3852,4202169CMAA003VE,2021-06-01,19
3512,3865,21040520000000GR,2021-05-01,408


In [7]:
pivot = df.pivot_table(values='Cnt_Sale', index=df.index, columns='Period', aggfunc='first')
table = pd.concat([df[['Id_Client', 'id_Product']], pivot], axis=1)

In [8]:
real_df = table.groupby(['Id_Client', 'id_Product']).sum()
real_df

2020-08-01 00:00:00  2020-09-01 00:00:00  \
Id_Client id_Product                                                   
6         2514400PCAO00100                  0.0                  0.0   
9         2513490PJFA00100                  0.0                  0.0   
36        1100032000000000                  0.0                  0.0   
          1100045000000000                  0.0                  0.0   
          1100093000000000                  0.0                  0.0   
...                                         ...                  ...   
3864      2514404PCCI00100                  0.0                  0.0   
3865      21040520000000GR                  0.0                  0.0   
3917      1450253FCAQ00200                  0.0                  0.0   
3939      2514404PCRD00100                  0.0                  0.0   
4034      41010510000000NE                  0.0                  0.0   

                            2020-10-01 00:00:00  2020-11-01 00:00:00  \
Id_Client id_Product                                                   
6         2514400PCAO00100                  0.0              15246.0   
9         2513490PJFA00100                  0.0                  0.0   
36        1100032000000000                150.0                  0.0   
          1100045000000000                  0.0                  0.0   
          1100093000000000                  0.0                  0.0   
...                                         ...                  ...   
3864      2514404PCCI00100                  0.0                  0.0   
3865      21040520000000GR                  0.0                  0.0   
3917      1450253FCAQ00200                  0.0                  0.0   
3939      2514404PCRD00100                  0.0                  0.0   
4034      41010510000000NE                  0.0                  0.0   

                            2020-12-01 00:00:00  2021-01-01 00:00:00  \
Id_Client id_Product                                                   
6         2514400PCAO00100                  0.0                  0.0   
9         2513490PJFA00100                  0.0                  0.0   
36        1100032000000000                300.0                  0.0   
          1100045000000000                 25.0                  0.0   
          1100093000000000                 75.0                  0.0   
...                                         ...                  ...   
3864      2514404PCCI00100                  0.0                  0.0   
3865      21040520000000GR                  0.0                  0.0   
3917      1450253FCAQ00200                  0.0                  0.0   
3939      2514404PCRD00100                  0.0                  0.0   
4034      41010510000000NE                  0.0                  0.0   

                            2021-02-01 00:00:00  2021-03-01 00:00:00  \
Id_Client id_Product                                                   
6         2514400PCAO00100                  0.0                  0.0   
9         2513490PJFA00100                  0.0               5370.0   
36        1100032000000000                  0.0                  0.0   
          1100045000000000                  0.0                  0.0   
          1100093000000000                  0.0                  0.0   
...                                         ...                  ...   
3864      2514404PCCI00100                  0.0                  0.0   
3865      21040520000000GR                  0.0                  0.0   
3917      1450253FCAQ00200                  0.0                  0.0   
3939      2514404PCRD00100                  0.0                  0.0   
4034      41010510000000NE                  0.0                  0.0   

                            2021-04-01 00:00:00  2021-05-01 00:00:00  \
Id_Client id_Product                                                   
6         2514400PCAO00100              15460.0                  0.0   
9         2513490PJFA00100                  0.0                 

#### NULL values are converted to 0 because the lack of these values means no sales 

In [9]:
real_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1382 entries, (6, '2514400PCAO00100') to (4034, '41010510000000NE')
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   2020-08-01 00:00:00  1382 non-null   float64
 1   2020-09-01 00:00:00  1382 non-null   float64
 2   2020-10-01 00:00:00  1382 non-null   float64
 3   2020-11-01 00:00:00  1382 non-null   float64
 4   2020-12-01 00:00:00  1382 non-null   float64
 5   2021-01-01 00:00:00  1382 non-null   float64
 6   2021-02-01 00:00:00  1382 non-null   float64
 7   2021-03-01 00:00:00  1382 non-null   float64
 8   2021-04-01 00:00:00  1382 non-null   float64
 9   2021-05-01 00:00:00  1382 non-null   float64
 10  2021-06-01 00:00:00  1382 non-null   float64
 11  2021-07-01 00:00:00  1382 non-null   float64
dtypes: float64(12)
memory usage: 146.1+ KB


In [10]:
real_df.describe(include='all')

,2020-08-01 00:00:00,2020-09-01 00:00:00,2020-10-01 00:00:00,2020-11-01 00:00:00,2020-12-01 00:00:00,2021-01-01 00:00:00,2021-02-01 00:00:00,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00,2021-06-01 00:00:00,2021-07-01 00:00:00
count,1382.000000,1382.000000,1382.000000,1382.000000,1382.000000,1382.000000,1382.000000,1382.000000,1382.000000,1382.000000,1382.000000,1382.000000
mean,3432.777135,3317.527496,3262.366136,3159.204052,2525.696816,3238.623010,3744.667873,2435.234443,2250.302460,2266.879161,2631.269899,2547.362518
std,13706.800360,11518.979024,12904.194326,14894.861009,8098.152157,10553.030224,15625.874356,10842.085017,7536.223721,8263.048359,8442.123217,8569.761939
min,-250.000000,-1500.000000,-18500.000000,-1000.000000,-12000.000000,-1.000000,-100.000000,-1000.000000,-19.000000,-206.000000,-5200.000000,-96.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,193.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,334000.000000,176000.000000,240000.000000,353796.000000,96103.000000,195300.000000,366792.000000,205302.000000,89651.000000,132000.000000,129531.000000,110000.000000


## ARIMA model :

In [19]:
def fit(data) :
    X = data.values
    X = X.astype('float32')
    train_size = int(len(X) * 0.90)
    train, test = X[0:train_size], X[train_size:]
    # walk-forward validation
    history = [x for x in train]
    predictions = list()
    for i in range(len(test)):
        # difference data
        months_in_year = 12
        #diff = difference(history, months_in_year)
        # predict
        model = ARIMA(history, order=(0,0,1))
        
        #model = arima_model.ARIMA(history, order=(0,0,1))
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        # observation
        obs = test[i]
        history.append(obs)
    # report performance
    rmse = sqrt(mean_squared_error(test, predictions))
    print('RMSE: %.3f' % rmse)
    # errors
    residuals = [test[i]-predictions[i] for i in range(len(test))]
    residuals = pd.DataFrame(residuals)
    print(residuals.describe())
    return rmse, yhat

In [20]:
preds = []
rmses = []

for idx in range(1382) :
    dd = pd.DataFrame(df.iloc[idx])
    rmse, yhat = fit(dd)
    preds.append(yhat)
    rmses.append(rmse)
  

RMSE: 2833.144
                 0
count     2.000000
mean  -2455.288757
std    1999.130790
min   -3868.887695
25%   -3162.088226
50%   -2455.288757
75%   -1748.489288
max   -1041.689819
RMSE: 608.233
                0
count    2.000000
mean  -607.682007
std     36.609521
min   -633.568848
25%   -620.625427
50%   -607.682007
75%   -594.738586
max   -581.795166
RMSE: 54.474
               0
count   2.000000
mean  -54.306501
std     6.033349
min   -58.572723
25%   -56.439612
50%   -54.306501
75%   -52.173390
max   -50.040279
RMSE: 3.006
              0
count  2.000000
mean  -2.997159
std    0.334817
min   -3.233910
25%   -3.115534
50%   -2.997159
75%   -2.878783
max   -2.760407
RMSE: 9.022
              0
count  2.000000
mean  -8.994628
std    1.000649
min   -9.702194
25%   -9.348411
50%   -8.994628
75%   -8.640846
max   -8.287063
RMSE: 177.257
                0
count    2.000000
mean  -146.004761
std    142.145381
min   -246.516724
25%   -196.260742
50%   -146.004761
75%    -95.748779
ma

In [21]:
resARIMA = pd.DataFrame(list(zip(preds, rmses)),
               columns =['predictions', 'RMSE'])

resARIMA.index = df.index
resARIMA.to_csv('ARIMA_result.csv')

resARIMA

predictions         RMSE
Id_Client id_Product                                
6         2514400PCAO00100  3868.887652  2833.143956
9         2513490PJFA00100   581.795177   608.233156
36        1100032000000000    50.040281    54.473817
          1100045000000000     2.760407     3.006495
          1100093000000000     8.287062     9.022416
...                                 ...          ...
3864      2514404PCCI00100    -0.000005  7631.803493
3865      21040520000000GR    37.140977    32.747460
3917      1450253FCAQ00200    18.801108    16.556794
3939      2514404PCRD00100    -0.000005  7395.629828
4034      41010510000000NE    -0.000005    42.426410

[1382 rows x 2 columns]

In [23]:
print('RMSE for ARIMA method is :', resARIMA.RMSE.mean())

RMSE for ARIMA method is : 4506.316100994384


## NPTS

In [136]:
# a function to create our training data and test data, then train the estimator and evaluate it
def NPTS(data) :
    training_data = ListDataset(
        [{"start": data.index[0], "target": data[data.columns[0]][:'2021-07-01 00:00:00']}],
        freq = "1M"
      )
    test_data = ListDataset(
        [{"start":  data[data.columns[0]]['2021-06-01 00:00:00'], "target":  data[data.columns[0]][:"2021-07-01 00:00:00"]}],
        freq = "1M"
      )

    npts_predictor = NPTSPredictor(freq="1M", prediction_length=1, context_length=12, kernel_type='uniform', use_seasonal_model=False)
    npts_forecast = list(npts_predictor.predict(training_data))#training_ds))

      #predictor = estimator.train(training_data=training_ds)#, num_workers=4)
    forecast_it, ts_it = make_evaluation_predictions(
          dataset=test_data, predictor=npts_predictor, num_samples=1
         )#test_ds

    forcasts = list(forecast_it)
    tss = list(ts_it)

    evaluator = Evaluator(quantiles=[0.25,0.5,0.75])

    npts_agg_metrics, npts_item_metrics = evaluator(
        iter(tss), iter(forcasts), num_series=len(test_data)
      )

    rmse = np.sqrt(npts_item_metrics['MSE'])[0]
    return rmse, forcasts[0].samples

In [137]:
preds = []
rmses = []

for idx in range(1382) :
    dd = pd.DataFrame(df.iloc[idx])
    rmse, yhat = NPTS(dd)
    preds.append(yhat)
    rmses.append(rmse)
    

resNPTS = pd.DataFrame(list(zip(preds, rmses)),
               columns =['predictions', 'RMSE'])

resNPTS.index = df.index

Running evaluation: 100%|██████████| 1/1 [00:00<00:00, 20.99it/s]


In [134]:
resNPTS.to_csv('NPTS_result.csv')
resNPTS.sample(5)

predictions     RMSE
Id_Client id_Product                           
6         2514400PCAO00100     [[0.0]]      0.0
9         2513490PJFA00100     [[0.0]]      0.0
36        1100032000000000     [[0.0]]      0.0
          1100045000000000     [[0.0]]      0.0
          1100093000000000     [[0.0]]      0.0
...                                ...      ...
3864      2514404PCCI00100     [[0.0]]  10793.0
3865      21040520000000GR     [[0.0]]      0.0
3917      1450253FCAQ00200     [[0.0]]      0.0
3939      2514404PCRD00100     [[0.0]]  10459.0
4034      41010510000000NE     [[0.0]]     60.0

[1382 rows x 2 columns]

In [133]:
resNPTS.RMSE.mean()

3877.6968162083936

## Predictions for the next three months :

In [4]:
arima= pd.read_csv('ARIMA_result.csv')
npts= pd.read_csv('NPTS_result.csv')

In [5]:
all_rmse = pd.concat([arima[['RMSE']].add_prefix('arima_'), npts[['RMSE']].add_prefix('npts_')], axis = 1)

In [6]:
def minRMSE() :
    mins = []
    for idx in range(1382) :
        min_ = all_rmse.iloc[idx].argmin()
        if min_ == 0 :
            mins.append('ARIMA')
        else :
            mins.append('NPTS')
    return mins
            
all_rmse['Best Model'] = minRMSE()

In [154]:
# let's create now a copy of our data and add three columns for the three next months and initialize it with 0
result = df
result['2021-08-01 00:00:00'] = 0
result['2021-09-01 00:00:00'] = 0
result['2021-10-01 00:00:00'] = 0
result

2020-08-01 00:00:00  ...  2021-10-01 00:00:00
Id_Client id_Product                             ...                     
6         2514400PCAO00100                  0.0  ...                    0
9         2513490PJFA00100                  0.0  ...                    0
36        1100032000000000                  0.0  ...                    0
          1100045000000000                  0.0  ...                    0
          1100093000000000                  0.0  ...                    0
...                                         ...  ...                  ...
3864      2514404PCCI00100                  0.0  ...                    0
3865      21040520000000GR                  0.0  ...                    0
3917      1450253FCAQ00200                  0.0  ...                    0
3939      2514404PCRD00100                  0.0  ...                    0
4034      41010510000000NE                  0.0  ...                    0

[1382 rows x 15 columns]

In [ ]:
# let's add the column with the best model to our dataframe result
result = pd.concat([result, all_rmse[['Best Model']]], axis = 1)

In [ ]:
# function to predict the sales for the next months
def nextMonthsARIMA(data) :
    X = data.values
    train = X.astype('float32')
    history = [x for x in train]
    predictions = list()
    for i in range(3) :#len(test)):
        model = ARIMA(history, order=(0,0,1))
        #model = arima_model.ARIMA(history, order=(0,0,1))
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        #yhat = model_fit.predict()
        predictions.append(yhat)
        obs = yhat
        history.append(obs)
    return predictions


In [157]:
# function to predict the sales for the next months
def nextMonthsNPTS(data) : 

    training_data = ListDataset(
      [{"start": data.index[0], "target": data[data.columns[0]][:"2021-08-01 00:00:00"]}],
      freq = "1M"
    )

    test_data = ListDataset(
      [{"start":  data[data.columns[0]]['2021-08-01 00:00:00'], "target":  data[data.columns[0]][:]}],
      freq = "1M"
    )
  

   print(training_data)

    npts_predictor = NPTSPredictor(freq="1M", prediction_length=3, context_length=12, kernel_type='uniform', use_seasonal_model=False)
    npts_forecast = list(npts_predictor.predict(training_data))#training_ds))

    #predictor = estimator.train(training_data=training_ds)#, num_workers=4)
    forecast_it, ts_it = make_evaluation_predictions(
          dataset=test_data, predictor=npts_predictor, num_samples=1
        )#test_ds
      
    forcasts = list(forecast_it)
    return forcasts[0].samples[0][0], forcasts[0].samples[0][1], forcasts[0].samples[0][2]

In [159]:
for idx in range(1382) :
    if result.iloc[idx]['Best Model'] == 'ARIMA' :
        M1, M2, M3 = nextMonthsARIMA(result.iloc[idx][2:-4])
    else :
        dd = pd.DataFrame(result.iloc[0][2:-1])
        M1, M2, M3 = nextMonthsNPTS(dd)
        
    result.iloc[idx, result.columns.get_loc('2021-08-01 00:00:00')] = M1
    result.iloc[idx, result.columns.get_loc('2021-09-01 00:00:00')] = M2
    result.iloc[idx, result.columns.get_loc('2021-10-01 00:00:00')] = M3
        

In [3]:
result

,Unnamed: 0,Id_Client,id_Product,2020-08-01 00:00:00,2020-09-01 00:00:00,2020-10-01 00:00:00,2020-11-01 00:00:00,2020-12-01 00:00:00,2021-01-01 00:00:00,2021-02-01 00:00:00,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00,2021-06-01 00:00:00,2021-07-01 00:00:00,2021-08-01 00:00:00,2021-09-01 00:00:00,2021-10-01 00:00:00,Best Model
0,0,6,2514400PCAO00100,0.0,0.0,0.0,15246.0,0.0,0.0,0.0,0.0,15460.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,NPTS
1,1,9,2513490PJFA00100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5370.0,0.0,0.0,0.0,0.0,0.000000,5370.000000,5370.000000,NPTS
2,2,36,1100032000000000,0.0,0.0,150.0,0.0,300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,300.000000,300.000000,NPTS
3,3,36,1100045000000000,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,NPTS
4,4,36,1100093000000000,0.0,0.0,0.0,0.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,NPTS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,1377,3864,2514404PCCI00100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10793.0,198.328403,844.052740,848.206373,ARIMA
1378,1378,3865,21040520000000GR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,408.0,0.0,0.0,408.000000,0.000000,0.000000,NPTS
1379,1379,3917,1450253FCAQ00200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,206.0,0.0,0.0,0.000000,0.000000,0.000000,NPTS
1380,1380,3939,2514404PCRD00100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10459.0,193.447620,818.026098,822.047701,ARIMA


In [ ]:
result.to_csv('newResult.csv')